In [7]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd 

In [8]:
#Ingresar a la página principal de la NFL
page = requests.get("https://www.nfl.com/stats/player-stats/")
#Pasarlo a formato soup
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
#Obtener los enlaces a cada pestaña del navbar
pestanhas=[x.a['href'] for x in soup.find("ul",{"class":"d3-o-tabs d3-o-tabs__nowrap"}).find_all('li')]
linksJugadores=[]
#Iterar para sacar los datos
for pestanha in pestanhas:
    statPage=requests.get("https://www.nfl.com"+pestanha)
    soup2 = BeautifulSoup(statPage.content, 'html.parser')
    table=soup2.find('table')
    #Obtener cada fila de la tabla de jugadores
    jugadores=table.find_all('tr')
    for jugador in jugadores:
        if(jugador.find('div',{'class':'d3-o-media-object__body'}) is None):continue
        else:
            #Agregar el link al perfil del jugador
            linksJugadores.append("https://www.nfl.com"+jugador.find('div',{'class':'d3-o-media-object__body'}).a['href'])

In [10]:
#Remover duplicados
linksJugadores=(list(set(linksJugadores)))

In [ ]:
#sacar en formato soup los datos del jugador
datosHTML=[]
for linkJugador in linksJugadores:
    pageL=requests.get(linkJugador)
    datosHTML.append(BeautifulSoup(pageL.content, 'html.parser'))

In [ ]:
datosJugadores=[]
for html in datosHTML:
    #Validar que el jugador este activo
    try:
        estado=html.find('h3',{"class":"nfl-c-player-header__roster-status nfl-c-player-header__roster-status--act nfl-u-hide-empty"}).text
    except:
        continue
    if(estado!="active"):continue
    #sacar la data de la pagina usando find
    nombre=html.find('h1',{"class":"nfl-c-player-header__title"}).text
    posandnumber=html.find('div',{"class":"nfl-c-player-header__player-data nfl-u-hide-empty"}).text.split('•')
    posicion=posandnumber[0].strip()
    numero=posandnumber[1].strip()
    equipo=html.find("div",{"class":"nfl-c-player-header__team nfl-u-hide-empty"})
    datosEsp=html.find_all("div",{"class":"nfl-c-player-info__value"})
    altura=datosEsp[0].text.strip()
    peso=int(datosEsp[1].text.strip())
    brazos=datosEsp[2].text.strip()
    manos=datosEsp[3].text.strip()
    experiencia=int(datosEsp[4].text.strip())
    instituto=datosEsp[5].text.strip()
    edad=int(datosEsp[6].text.strip())
    ciudad=datosEsp[7].text.strip()
    datosJugadores.append([nombre,posicion,numero,altura,peso,brazos,manos,experiencia,instituto,edad,ciudad])
dfDatos=pd.DataFrame(datosJugadores,columns=["Nombre","Posicion","Numero","Altura","Peso","Brazos","Manos","Experiencia","Instituto","Edad","Ciudad"])
dfDatos